In [ ]:
---
layout: post
title: The Case For Decentralization
description: What is decentralization? And why would I use it over centralized services?
permalink: /digital-famine/microblog/decentralization
breadcrumb: true
---

## Introduction

You can't and *shouldn't* trust massive, centralized tech conglomerates at all. The truth is that using any massive tech company and trusting them with any of your data is a 99.9% of the time a bad move. There's **2 main arguments** for this:

1. Big tech companies collect a lot of data about *you*, that comes with its own plethora of problems; namely, the main problem is the amount of tracking they include in their profit models. See, Google and Meta (for example) make most of their money off of you by brokering your data (collecting and selling it) and using it to target you with... advertisements! The modern society is all *consume*, *consume*, *consume*, and they make a HUGE amount of money off of it. As such, your data is incredibly valuable, and these companies have a [very common](https://haveibeenpwned.com/) tendency to leak that data. Since these centralized models focus on surveillance and selling your data (if that isn't a massive concern already as it paves way for profits over humanity), it can also be really easily abused to control people, bringing us to the next point;

2. Because basically everybody you know uses these centralized platforms, they're tied into said systems and thus reliant on a frankly parasitic relationship, making a perfect mass surveillance model. Governments and companies alike have abused this model to monitor, oppress, and control people in clear violation of legal and constitutional protections. Here's some links:
   - [Electronic Frontier Foundation](https://www.eff.org/)
   - [Global surveillance Wikipedia](https://en.wikipedia.org/wiki/Global_surveillance)
   - [Vault 7 (CIA Hacking Tools Revealed)](https://wikileaks.org/ciav7p1/)
   - [Edward Snowden's Memoir](https://avalonlibrary.net/ebooks/Edward%20Snowden%20-%20Permanent%20Record.pdf)
   - [Edward Snowden's Wikipedia](https://en.wikipedia.org/wiki/Edward_Snowden)

## Trust is a Funny Thing

Here's an example. Assume that I'm a massive tech conglomerate (like Google) and you want to store your files in a very convenient service which saves your files in cloud infrastructure (like Google Drive). Now, using my proprietary service, you can't really know how I'm handling your data, you just have to place your trust in me. (Side note: you can't even trust open source services at times because you don't always know if their servers run that specific code or different code, but in most cases it is *much* better than using large, centralized services.) Now, let's assume that I use some insecure form of encryption, like `AES 128 ECB`.

You can see penguins through it!

![ECB Penguins](https://i.sstatic.net/bXAUL.png)